
Here is the implementation of the **GCHM** model, complete with all the necessary functions. This shell script serves as the backbone for running the model. You can access everything via the following GDrive link: [GCHM_New_Archi](https://drive.google.com/drive/folders/1hNXEb6LdKN2QwD_VMlSGuphRrODkqYKy?usp=drive_link)

### **Important Notes:**


*  **GPU Required:** Make sure your system is equipped with a GPU to run the model effectively.

*   **Folder Placement:** Once you download the folder, place it in /content/drive/MyDrive. It should work seamlessly from there. If you encounter any issues with file paths, you’ll need to modify them in the following files: *deploy.py, config.json, and ee_preprocess.py.*

*   **Test Data:** You already have my previous test runs on the drive (Greece 2020 and Greece 2024), which can be visualized using QGIS.

*   **GeoJSON Shape Requirements:**
To run it on your AoI, just put the GeoJSON file, and adapt the path in config.json
Ensure that your GeoJSON file defines a perfect square for the Area of Interest (AoI). If the shape isn’t square, the CNN will break due to issues with batch processing. To solve this, I usually select an AoI on geojson.io and ask ChatGPT to adjust it into a perfect square.









Mount the drive to access the folder

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install all the libraries required

In [1]:
!pip install -r /content/drive/MyDrive/GCHM_New_Archi/requirements.txt

Connect to Earth Engine ! You’ll also need to set up your Earth Engine project and API credentials. Alternatively, you can use mine: /content/drive/MyDrive/GCHM_New_Archi/ee-crampemarc-fe52ce4721a3.json.

In [17]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3SQQYkv0Cm4bVWUHZm_tQsXXd8xUP3Xxx2VhSG7jjvE&tc=BSHFbLaUHVAKkszS9GwJ5K5bg3Dh6JqQEbz46ijvVIM&cc=wYQNzZoLsnKYtRU6bG6S0FZSRO8jXMbB9Q0uGttk1Ko

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASVgi3LkOV9b1pya722Qq5HKorREUWn0_Pi3gOIXhBewBj91RA1RTSD963U

Successfully saved authorization token.


This part download the GeoTIF from GEE, on your selected AoI and dates in *config.json*

In [50]:
%%bash

CONFIG_PATH="/content/drive/MyDrive/GCHM_New_Archi/config.json"
GEE_SCRIPT="/content/drive/MyDrive/GCHM_New_Archi/ee_preprocess.py"

echo "Running the GEE export process..."
python3 "$GEE_SCRIPT" --config "$CONFIG_PATH"

# Vérifier si le script GEE a réussi
if [ $? -ne 0 ]; then
  echo "GEE script failed. Exiting..."
  exit 1
fi

echo "All images have been successfully exported from GEE!"

Running the GEE export process...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Exporting Greece_2020-06-01...
Export of Greece_2020-06-01 completed.
Image is saved in Google Drive at: Dataset_GEE_TIF/Greece_2020-06-01.tif
Exporting Greece_2024-06-01...
Exporting Greece_2024-06-01...
Exporting Greece_2024-06-01...
Exporting Greece_2024-06-01...
Exporting Greece_

Run the GCHM model trough your GeoTIF you just downloaded

In [51]:
%%bash

GCHM_MODEL_DIR="/content/drive/MyDrive/GCHM_New_Archi/trained_models/GLOBAL_GEDI_2019_2020"
DEPLOY_IMAGE_PATH="/content/drive/MyDrive/GCHM_New_Archi/Dataset_GEE_TIF"
GCHM_DEPLOY_DIR="/content/drive/MyDrive/GCHM_New_Archi/Predictions"
FAIL_DIR="/content/drive/MyDrive/GCHM_New_Archi/Log_Failed/log_failed.txt"
S2_DIR="/content/drive/MyDrive/GCHM_New_Archi/Dataset_GEE_TIF"

# create directories
mkdir -p ${GCHM_DEPLOY_DIR}

echo "Running the deployment command for all images..."
count=1
for deploy_image in "$DEPLOY_IMAGE_PATH"/*; do
  if [[ -f "$deploy_image" ]]; then  # Vérifier si c'est un fichier
    echo "*************************************"
    echo "Processing image ${count}: $deploy_image"

    # Exécuter la commande de déploiement
    python3 /content/drive/MyDrive/GCHM_New_Archi/gchm/deploy.py \
      --model_dir="${GCHM_MODEL_DIR}" \
      --deploy_image_path="${deploy_image}" \
      --deploy_dir="${GCHM_DEPLOY_DIR}" \
      --deploy_patch_size=512 \
      --num_workers_deploy=4 \
      --num_models=5 \
      --finetune_strategy="FT_Lm_SRCB" \
      --filepath_failed_image_paths="${FAIL_DIR}" \
      --sentinel2_dir="${S2_DIR}" \
      --remove_image_after_pred="False"

    # Vérifier si la commande de déploiement a réussi
    exit_status=$?  # Stocker le code de sortie
    if [ $exit_status -ne 0 ]; then
      echo "Deployment failed for image $deploy_image. Logging failed image."
      echo "$deploy_image" >> "$FAIL_DIR"
    fi

    count=$((count + 1))
  fi
done

echo "All processes completed successfully!"

Running the deployment command for all images...
*************************************
Processing image 1: /content/drive/MyDrive/GCHM_New_Archi/Dataset_GEE_TIF/Greece_2020-06-01.tif
DEVICE:  cuda:0 NVIDIA L4
Sampled model_id: 4 out of 5 models in ensemble.
Using args.model_dir:  /content/drive/MyDrive/GCHM_New_Archi/trained_models/GLOBAL_GEDI_2019_2020/model_4/FT_Lm_SRCB
Loading args from trained models directory...
{'model_dir': '/content/drive/MyDrive/GCHM_New_Archi/trained_models/GLOBAL_GEDI_2019_2020/model_4/FT_Lm_SRCB', 'deploy_image_path': '/content/drive/MyDrive/GCHM_New_Archi/Dataset_GEE_TIF/Greece_2020-06-01.tif', 'deploy_dir': '/content/drive/MyDrive/GCHM_New_Archi/Predictions', 'filepath_failed_image_paths': '/content/drive/MyDrive/GCHM_New_Archi/Log_Failed/log_failed.txt', 'deploy_patch_size': 512, 'deploy_batch_size': 2, 'num_workers_deploy': 4, 'num_models': 5, 'save_latlon_masks': False, 'remove_image_after_pred': False, 'sentinel2_dir': '/content/drive/MyDrive/GCHM_New

/content/drive/MyDrive/GCHM_New_Archi/gchm/deploy.py:219: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)
/content/drive/MyDrive/GCHM_

____________


You should find your results in : /content/drive/MyDrive/GCHM_New_Archi/Predictions/
